In [ ]:
! pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.5 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForTokenClassification, AdamW
from sklearn.metrics import classification_report

In [ ]:
ner_data = [
    ("A NANA KWESI SARPONG. 14YRS", [("NANA KWESI SARPONG", "NAME"), ("14YRS", "AGE")]),
    ("THERESA,KOOMSON,70YRS L-P A AJSPECIALISTSCAN Ref:/Perf ID:FEB Study date: 16 F Study time: 08:09:48 4cm W1024 / C512 CHEST 1001 IMA 1001 S-Value: 2198.000 Zoom factor: x0.99", [("THERESA KOOMSON", "NAME"), ("70YRS", "AGE"), ("F", "SEX")]),
    ("JOHN DOE, 45 YEARS OLD, MALE", [("JOHN DOE", "NAME"), ("45", "AGE"), ("MALE", "SEX")]),
    ("A. Johanssen. Hosp no. 012345 Date. Today", [("A. Johanssen", "NAME")]),
    ("CHARLES,OBIMPEH,MENSAH,,38YRS Per 4cm W1024 / C512 CHEST Pos 1001 IMA 1001 S-Value: 449.000 Zoom factor", [("CHARLES, OBIMPEH, MENSAH", "NAME"), ("38YRS", "AGE")]),
    ("SARAH, OWUSU, 25 YEARS OLD, FEMALE", [("SARAH OWUSU", "NAME"), ("25", "AGE"), ("FEMALE", "SEX")]),
    ("KOFI ANTWI, 32 YEARS OLD, MALE", [("KOFI ANTWI", "NAME"), ("32", "AGE"), ("MALE", "SEX")]),
    ("EMILY ADDO, 28YRS", [("EMILY ADDO", "NAME"), ("28YRS", "AGE")]),
    ("MATTHEW ASANTE, 37 YEARS OLD, MALE", [("MATTHEW ASANTE", "NAME"), ("37", "AGE"), ("MALE", "SEX")]),
    ("LUCY OSEI, 42 YEARS OLD, FEMALE", [("LUCY OSEI", "NAME"), ("42", "AGE"), ("FEMALE", "SEX")]),
    ("DANIEL BOATENG, 55YRS M", [("DANIEL BOATENG", "NAME"), ("55YRS", "AGE"), ("M", "SEX")]),
    ("JESSICA AMPONSAH, 29 YEARS OLD, FEMALE", [("JESSICA AMPONSAH", "NAME"), ("29", "AGE"), ("FEMALE", "SEX")]),
    ("KWADWO MENSAH, 45YRS", [("KWADWO MENSAH", "NAME"), ("45YRS", "AGE")]),
    ("IRENE OWUSU, 39 YEARS OLD, FEMALE", [("IRENE OWUSU", "NAME"), ("39", "AGE"), ("FEMALE", "SEX")]),
    ("ERIC ADU, 33YRS M", [("ERIC ADU", "NAME"), ("33YRS", "AGE"), ("M", "SEX")]),
    ("RACHEL AMOAH, 49 YEARS OLD, FEMALE", [("RACHEL AMOAH", "NAME"), ("49", "AGE"), ("FEMALE", "SEX")]),
    ("KWAME NTI, 24YRS", [("KWAME NTI", "NAME"), ("24YRS", "AGE")]),
    ("ELIZABETH ANSAH, 51 YEARS OLD, FEMALE", [("ELIZABETH ANSAH", "NAME"), ("51", "AGE"), ("FEMALE", "SEX")]),
    ("ISAAC MENSAH, 30YRS M", [("ISAAC MENSAH", "NAME"), ("30YRS", "AGE"), ("M", "SEX")]),
    ("ABENA YEBOAH, 26 YEARS OLD, FEMALE", [("ABENA YEBOAH", "NAME"), ("26", "AGE"), ("FEMALE", "SEX")]),
    ("KOFI APPIAH, 43YRS", [("KOFI APPIAH", "NAME"), ("43YRS", "AGE")]),
    ("ESTHER OPOKU, 36 YEARS OLD, FEMALE", [("ESTHER OPOKU", "NAME"), ("36", "AGE"), ("FEMALE", "SEX")]),
    ("MICHAEL AMANKWAH, 22YRS M", [("MICHAEL AMANKWAH", "NAME"), ("22YRS", "AGE"), ("M", "SEX")]),
    ("REBECCA ASANTE, 58 YEARS OLD, FEMALE", [("REBECCA ASANTE", "NAME"), ("58", "AGE"), ("FEMALE", "SEX")]),
    ("KWASI OSEI, 27YRS", [("KWASI OSEI", "NAME"), ("27YRS", "AGE")]),
    ("RITA MENSAH, 49 YEARS OLD, FEMALE", [("RITA MENSAH", "NAME"), ("49", "AGE"), ("FEMALE", "SEX")]),
    ("FRANCIS BOAKYE, 31YRS M", [("FRANCIS BOAKYE", "NAME"), ("31YRS", "AGE"), ("M", "SEX")]),
    ("AGNES OPOKU, 53 YEARS OLD, FEMALE", [("AGNES OPOKU", "NAME"), ("53", "AGE"), ("FEMALE", "SEX")]),
    ("KWAME MENSAH, 29YRS", [("KWAME MENSAH", "NAME"), ("29YRS", "AGE")]),
    ("VICTORIA AMOAH, 47 YEARS OLD, FEMALE", [("VICTORIA AMOAH", "NAME"), ("47", "AGE"), ("FEMALE", "SEX")]),
    ("ANDREW AKUFFO, 25YRS M", [("ANDREW AKUFFO", "NAME"), ("25YRS", "AGE"), ("M", "SEX")]),
    ("EVELYN YEBOAH, 60 YEARS OLD, FEMALE", [("EVELYN YEBOAH", "NAME"), ("60", "AGE"), ("FEMALE", "SEX")]),
    ("CHARLES OBIMPEH MENSAH, 38YRS Per 4cm W1024 / C512 CHEST Pos 1001 IMA 1001 S-Value: 449.000 Zoom factor", [("CHARLES OBIMPEH MENSAH", "NAME"), ("38YRS", "AGE")]),
    ("SARAH OWUSU, 25 YEARS OLD, FEMALE", [("SARAH OWUSU", "NAME"), ("25", "AGE"), ("FEMALE", "SEX")]),
    ("A NANA KWESI SARPONG, 14YRS", [("NANA KWESI SARPONG", "NAME"), ("14YRS", "AGE")]),
    ("THERESA KOOMSON, 70YRS L-P A AJSPECIALISTSCAN Ref:/Perf ID:FEB Study date: 16 F Study time: 08:09:48 4cm W1024 / C512 CHEST 1001 IMA 1001 S-Value: 2198.000 Zoom factor: x0.99", [("THERESA KOOMSON", "NAME"), ("70YRS", "AGE"), ("F", "SEX")]),
    ("JOHN DOE, 45 YEARS OLD, MALE", [("JOHN DOE", "NAME"), ("45", "AGE"), ("MALE", "SEX")]),
    ("A. Johanssen. Hosp no. 012345 Date. Today", [("A. Johanssen", "NAME")]),
    ("CHARLES OBIMPEH MENSAH, 38YRS Per 4cm W1024 / C512 CHEST Pos 1001 IMA 1001 S-Value: 449.000 Zoom factor", [("CHARLES OBIMPEH MENSAH", "NAME"), ("38YRS", "AGE")]),
    ("SARAH OWUSU, 25 YEARS OLD, FEMALE", [("SARAH OWUSU", "NAME"), ("25", "AGE"), ("FEMALE", "SEX")]),
    ("JAMES KING, 50YRS M L-P A", [("JAMES KING", "NAME"), ("50YRS", "AGE"), ("M", "SEX")]),
    ("JULIA ANDERSON, 32 YEARS OLD, FEMALE", [("JULIA ANDERSON", "NAME"), ("32", "AGE"), ("FEMALE", "SEX")]),
    ("DAVID WHITE, 43YRS M L-P A", [("DAVID WHITE", "NAME"), ("43YRS", "AGE"), ("M", "SEX")]),
    ("EMILY JONES, 28 YEARS OLD, FEMALE", [("EMILY JONES", "NAME"), ("28", "AGE"), ("FEMALE", "SEX")]),
    ("MICHAEL BROWN, 37YRS M L-P A", [("MICHAEL BROWN", "NAME"), ("37YRS", "AGE"), ("M", "SEX")]),
    ("EVELYN YEBOAH, 60 YEARS OLD, FEMALE", [("EVELYN YEBOAH", "NAME"), ("60", "AGE"), ("FEMALE", "SEX")]),
    ("CHARLES OBIMPEH MENSAH, 38YRS Per 4cm W1024 / C512 CHEST Pos 1001 IMA 1001 S-Value: 449.000 Zoom factor", [("CHARLES OBIMPEH MENSAH", "NAME"), ("38YRS", "AGE")]),
    ("SARAH OWUSU, 25 YEARS OLD, FEMALE", [("SARAH OWUSU", "NAME"), ("25", "AGE"), ("FEMALE", "SEX")]),
    ("A NANA KWESI SARPONG, 14YRS", [("NANA KWESI SARPONG", "NAME"), ("14YRS", "AGE")]),
    ("THERESA KOOMSON, 70YRS L-P A AJSPECIALISTSCAN Ref:/Perf ID:FEB Study date: 16 F Study time: 08:09:48 4cm W1024 / C512 CHEST 1001 IMA 1001 S-Value: 2198.000 Zoom factor: x0.99", [("THERESA KOOMSON", "NAME"), ("70YRS", "AGE"), ("F", "SEX")]),
    ("JOHN DOE, 45 YEARS OLD, MALE", [("JOHN DOE", "NAME"), ("45", "AGE"), ("MALE", "SEX")]),
    ("A. Johanssen. Hosp no. 012345 Date. Today", [("A. Johanssen", "NAME")]),
    ("CHARLES OBIMPEH MENSAH, 38YRS Per 4cm W1024 / C512 CHEST Pos 1001 IMA 1001 S-Value: 449.000 Zoom factor", [("CHARLES OBIMPEH MENSAH", "NAME"), ("38YRS", "AGE")]),
    ("SARAH OWUSU, 25 YEARS OLD, FEMALE", [("SARAH OWUSU", "NAME"), ("25", "AGE"), ("FEMALE", "SEX")]),
    ("JAMES KING, 50YRS M L-P A", [("JAMES KING", "NAME"), ("50YRS", "AGE"), ("M", "SEX")]),
    ("JULIA ANDERSON, 32 YEARS OLD, FEMALE", [("JULIA ANDERSON", "NAME"), ("32", "AGE"), ("FEMALE", "SEX")]),
    ("DAVID WHITE, 43YRS M L-P A", [("DAVID WHITE", "NAME"), ("43YRS", "AGE"), ("M", "SEX")]),
    ("EMILY JONES, 28 YEARS OLD, FEMALE", [("EMILY JONES", "NAME"), ("28", "AGE"), ("FEMALE", "SEX")]),
    ("MICHAEL BROWN, 37YRS M L-P A", [("MICHAEL BROWN", "NAME"), ("37YRS", "AGE"), ("M", "SEX")]),
    ("EVELYN YEBOAH, 60 YEARS OLD, FEMALE", [("EVELYN YEBOAH", "NAME"), ("60", "AGE"), ("FEMALE", "SEX")]),
    ("CHARLES OBIMPEH MENSAH, 38YRS Per 4cm W1024 / C512 CHEST Pos 1001 IMA 1001 S-Value: 449.000 Zoom factor", [("CHARLES OBIMPEH MENSAH", "NAME"), ("38YRS", "AGE")]),
    ("SARAH OWUSU, 25 YEARS OLD, FEMALE", [("SARAH OWUSU", "NAME"), ("25", "AGE"), ("FEMALE", "SEX")]),
    ("JAMES KING, 50YRS M L-P A", [("JAMES KING", "NAME"), ("50YRS", "AGE"), ("M", "SEX")]),
    ("JULIA ANDERSON, 32 YEARS OLD, FEMALE", [("JULIA ANDERSON", "NAME"), ("32", "AGE"), ("FEMALE", "SEX")]),
    ("DAVID WHITE, 43YRS M L-P A", [("DAVID WHITE", "NAME"), ("43YRS", "AGE"), ("M", "SEX")]),
    ("EMILY JONES, 28 YEARS OLD, FEMALE", [("EMILY JONES", "NAME"), ("28", "AGE"), ("FEMALE", "SEX")]),
    ("MICHAEL BROWN, 37YRS M L-P A", [("MICHAEL BROWN", "NAME"), ("37YRS", "AGE"), ("M", "SEX")]),
    ("EVELYN YEBOAH, 60 YEARS OLD, FEMALE", [("EVELYN YEBOAH", "NAME"), ("60", "AGE"), ("FEMALE", "SEX")]),
    ("CHARLES OBIMPEH MENSAH, 38YRS Per 4cm W1024 / C512 CHEST Pos 1001 IMA 1001 S-Value: 449.000 Zoom factor", [("CHARLES OBIMPEH MENSAH", "NAME"), ("38YRS", "AGE")]),
    ("SARAH OWUSU, 25 YEARS OLD, FEMALE", [("SARAH OWUSU", "NAME"), ("25", "AGE"), ("FEMALE", "SEX")]),
    ("A NANA KWESI SARPONG, 14YRS", [("NANA KWESI SARPONG", "NAME"), ("14YRS", "AGE")]),
    ("THERESA KOOMSON, 70YRS L-P A AJSPECIALISTSCAN Ref:/Perf ID:FEB Study date: 16 F Study time: 08:09:48 4cm W1024 / C512 CHEST 1001 IMA 1001 S-Value: 2198.000 Zoom factor: x0.99", [("THERESA KOOMSON", "NAME"), ("70YRS", "AGE"), ("F", "SEX")]),
    ("JOHN DOE, 45 YEARS OLD, MALE", [("JOHN DOE", "NAME"), ("45", "AGE"), ("MALE", "SEX")]),
    ("A. Johanssen. Hosp no. 012345 Date. Today", [("A. Johanssen", "NAME")]),
    ("CHARLES OBIMPEH MENSAH, 38YRS Per 4cm W1024 / C512 CHEST Pos 1001 IMA 1001 S-Value: 449.000 Zoom factor", [("CHARLES OBIMPEH MENSAH", "NAME"), ("38YRS", "AGE")]),
    ("SARAH OWUSU, 25 YEARS OLD, FEMALE", [("SARAH OWUSU", "NAME"), ("25", "AGE"), ("FEMALE", "SEX")]),
    ("JAMES KING, 50YRS M L-P A", [("JAMES KING", "NAME"), ("50YRS", "AGE"), ("M", "SEX")]),
    ("JULIA ANDERSON, 32 YEARS OLD, FEMALE", [("JULIA ANDERSON", "NAME"), ("32", "AGE"), ("FEMALE", "SEX")]),
    ("DAVID WHITE, 43YRS M L-P A", [("DAVID WHITE", "NAME"), ("43YRS", "AGE"), ("M", "SEX")]),
    ("EMILY JONES, 28 YEARS OLD, FEMALE", [("EMILY JONES", "NAME"), ("28", "AGE"), ("FEMALE", "SEX")]),
    ("MICHAEL BROWN, 37YRS M L-P A", [("MICHAEL BROWN", "NAME"), ("37YRS", "AGE"), ("M", "SEX")]),
    ("EVELYN YEBOAH, 60 YEARS OLD, FEMALE", [("EVELYN YEBOAH", "NAME"), ("60", "AGE"), ("FEMALE", "SEX")]),
    ("CHARLES OBIMPEH MENSAH, 38YRS Per 4cm W1024 / C512 CHEST Pos 1001 IMA 1001 S-Value: 449.000 Zoom factor", [("CHARLES OBIMPEH MENSAH", "NAME"), ("38YRS", "AGE")]),
    ("SARAH OWUSU, 25 YEARS OLD, FEMALE", [("SARAH OWUSU","NAME"), ("25", "AGE"), ("FEMALE", "SEX")]),
    ("JAMES KING, 50YRS M L-P A", [("JAMES KING", "NAME"), ("50YRS", "AGE"), ("M", "SEX")]),
    ("JULIA ANDERSON, 32 YEARS OLD, FEMALE", [("JULIA ANDERSON", "NAME"), ("32", "AGE"), ("FEMALE", "SEX")]),
    ("DAVID WHITE, 43YRS M L-P A", [("DAVID WHITE", "NAME"), ("43YRS", "AGE"), ("M", "SEX")]),
    ("EMILY JONES, 28 YEARS OLD, FEMALE", [("EMILY JONES", "NAME"), ("28", "AGE"), ("FEMALE", "SEX")]),
    ("MICHAEL BROWN, 37YRS M L-P A", [("MICHAEL BROWN", "NAME"), ("37YRS", "AGE"), ("M", "SEX")]),
    ("EVELYN YEBOAH, 60 YEARS OLD, FEMALE", [("EVELYN YEBOAH", "NAME"), ("60", "AGE"), ("FEMALE", "SEX")]),
    ("CHARLES OBIMPEH MENSAH, 38YRS Per 4cm W1024 / C512 CHEST Pos 1001 IMA 1001 S-Value: 449.000 Zoom factor", [("CHARLES OBIMPEH MENSAH", "NAME"), ("38YRS", "AGE")]),
    ("SARAH OWUSU, 25 YEARS OLD, FEMALE", [("SARAH OWUSU", "NAME"), ("25", "AGE"), ("FEMALE", "SEX")]),
    ("A NANA KWESI SARPONG, 14YRS", [("NANA KWESI SARPONG", "NAME"), ("14YRS", "AGE")]),
    ("THERESA KOOMSON, 70YRS L-P A AJSPECIALISTSCAN Ref:/Perf ID:FEB Study date: 16 F Study time: 08:09:48 4cm W1024 / C512 CHEST 1001 IMA 1001 S-Value: 2198.000 Zoom factor: x0.99", [("THERESA KOOMSON", "NAME"), ("70YRS", "AGE"), ("F", "SEX")]),
    ("JOHN DOE, 45 YEARS OLD, MALE", [("JOHN DOE", "NAME"), ("45", "AGE"), ("MALE", "SEX")]),
    ("A. Johanssen. Hosp no. 012345 Date. Today", [("A. Johanssen", "NAME")]),
    ("CHARLES OBIMPEH MENSAH, 38YRS Per 4cm W1024 / C512 CHEST Pos 1001 IMA 1001 S-Value: 449.000 Zoom factor", [("CHARLES OBIMPEH MENSAH", "NAME"), ("38YRS", "AGE")]),
    ("SARAH OWUSU, 25 YEARS OLD, FEMALE", [("SARAH OWUSU", "NAME"), ("25", "AGE"), ("FEMALE", "SEX")]),
    ("JAMES KING, 50YRS M L-P A", [("JAMES KING", "NAME"), ("50YRS", "AGE"), ("M", "SEX")]),
    ("JULIA ANDERSON, 32 YEARS OLD, FEMALE", [("JULIA ANDERSON", "NAME"), ("32", "AGE"), ("FEMALE", "SEX")]),
    ("DAVID WHITE, 43YRS M L-P A", [("DAVID WHITE", "NAME"), ("43YRS", "AGE"), ("M", "SEX")]),
    ("EMILY JONES, 28 YEARS OLD, FEMALE", [("EMILY JONES", "NAME"), ("28", "AGE"), ("FEMALE", "SEX")]),
    ("MICHAEL BROWN, 37YRS M L-P A", [("MICHAEL BROWN", "NAME"), ("37YRS", "AGE"), ("M", "SEX")]),
    ("EVELYN YEBOAH, 60 YEARS OLD, FEMALE", [("EVELYN YEBOAH", "NAME"), ("60", "AGE"), ("FEMALE", "SEX")]),
    ("CHARLES OBIMPEH MENSAH, 38YRS Per 4cm W1024 / C512 CHEST Pos 1001 IMA 1001 S-Value: 449.000 Zoom factor", [("CHARLES OBIMPEH MENSAH", "NAME"), ("38YRS", "AGE")]),
    ("SARAH OWUSU, 25 YEARS OLD, FEMALE", [("SARAH OWUSU", "NAME"), ("25", "AGE"), ("FEMALE", "SEX")]),
    ("JAMES KING, 50YRS M L-P A", [("JAMES KING", "NAME"), ("50YRS", "AGE"), ("M", "SEX")]),
    ("JULIA ANDERSON, 32 YEARS OLD, FEMALE", [("JULIA ANDERSON", "NAME"), ("32", "AGE"), ("FEMALE", "SEX")]),
    ("DAVID WHITE, 43YRS M L-P A", [("DAVID WHITE", "NAME"), ("43YRS", "AGE"), ("M", "SEX")]),
    ("EMILY JONES, 28 YEARS OLD, FEMALE", [("EMILY JONES", "NAME"), ("28", "AGE"), ("FEMALE", "SEX")]),
    ("MICHAEL BROWN, 37YRS M L-P A", [("MICHAEL BROWN", "NAME"), ("37YRS", "AGE"), ("M", "SEX")]),
    ("EVELYN YEBOAH, 60 YEARS OLD, FEMALE", [("EVELYN YEBOAH", "NAME"), ("60", "AGE"), ("FEMALE", "SEX")]),
    ("CHARLES OBIMPEH MENSAH, 38YRS Per 4cm W1024 / C512 CHEST Pos 1001 IMA 1001 S-Value: 449.000 Zoom factor", [("CHARLES OBIMPEH MENSAH", "NAME"), ("38YRS", "AGE")]),
    ("SARAH OWUSU, 25 YEARS OLD, FEMALE", [("SARAH OWUSU", "NAME"), ("25", "AGE"), ("FEMALE", "SEX")]),
    ("A NANA KWESI SARPONG, 14YRS", [("NANA KWESI SARPONG", "NAME"), ("14YRS", "AGE")]),
    ("THERESA KOOMSON, 70YRS L-P A AJSPECIALISTSCAN Ref:/Perf ID:FEB Study date: 16 F Study time: 08:09:48 4cm W1024 / C512 CHEST 1001 IMA 1001 S-Value: 2198.000 Zoom factor: x0.99", [("THERESA KOOMSON", "NAME"), ("70YYRS", "AGE"), ("F", "SEX")]),
    ("JOHN DOE, 45 YEARS OLD, MALE", [("JOHN DOE", "NAME"), ("45", "AGE"), ("MALE", "SEX")]),
    ("A. Johanssen. Hosp no. 012345 Date. Today", [("A. Johanssen", "NAME")]),
    ("CHARLES OBIMPEH MENSAH, 38YRS Per 4cm W1024 / C512 CHEST Pos 1001 IMA 1001 S-Value: 449.000 Zoom factor", [("CHARLES OBIMPEH MENSAH", "NAME"), ("38YRS", "AGE")]),
    ("SARAH OWUSU, 25 YEARS OLD, FEMALE", [("SARAH OWUSU", "NAME"), ("25", "AGE"), ("FEMALE", "SEX")]),
    ("JAMES KING, 50YRS M L-P A", [("JAMES KING", "NAME"), ("50YRS", "AGE"), ("M", "SEX")]),
    ("JULIA ANDERSON, 32 YEARS OLD, FEMALE", [("JULIA ANDERSON", "NAME"), ("32", "AGE"), ("FEMALE", "SEX")]),
    ("DAVID WHITE, 43YRS M L-P A", [("DAVID WHITE", "NAME"), ("43YRS", "AGE"), ("M", "SEX")]),
    ("EMILY JONES, 28 YEARS OLD, FEMALE", [("EMILY JONES", "NAME"), ("28", "AGE"), ("FEMALE", "SEX")]),
    ("MICHAEL BROWN, 37YRS M L-P A", [("MICHAEL BROWN", "NAME"), ("37YRS", "AGE"), ("M", "SEX")]),
    ("EVELYN YEBOAH, 60 YEARS OLD, FEMALE", [("EVELYN YEBOAH", "NAME"), ("60", "AGE"), ("FEMALE", "SEX")]),
    ("CHARLES OBIMPEH MENSAH, 38YRS Per 4cm W1024 / C512 CHEST Pos 1001 IMA 1001 S-Value: 449.000 Zoom factor", [("CHARLES OBIMPEH MENSAH", "NAME"), ("38YRS", "AGE")]),
    ("SARAH OWUSU, 25 YEARS OLD, FEMALE", [("SARAH OWUSU", "NAME"), ("25", "AGE"), ("FEMALE", "SEX")]),
    ("A NANA KWESI SARPONG, 14YRS", [("NANA KWESI SARPONG", "NAME"), ("14YRS", "AGE")]),
    ("THERESA KOOMSON, 70YRS L-P A AJSPECIALISTSCAN Ref:/Perf ID:FEB Study date: 16 F Study time: 08:09:48 4cm W1024 / C512 CHEST 1001 IMA 1001 S-Value: 2198.000 Zoom factor: x0.99", [("THERESA KOOMSON", "NAME"), ("70YRS", "AGE"), ("F", "SEX")]),
    ("JOHN DOE, 45 YEARS OLD, MALE", [("JOHN DOE", "NAME"), ("45", "AGE"), ("MALE", "SEX")]),
    ("A. Johanssen. Hosp no. 012345 Date. Today", [("A. Johanssen", "NAME")]),
    ("CHARLES OBIMPEH MENSAH, 38YRS Per 4cm W1024 / C512 CHEST Pos 1001 IMA 1001 S-Value: 449.000 Zoom factor", [("CHARLES OBIMPEH MENSAH", "NAME"), ("38YRS", "AGE")]),
    ("SARAH OWUSU, 25 YEARS OLD, FEMALE", [("SARAH OWUSU", "NAME"), ("25", "AGE"), ("FEMALE", "SEX")]),
    ("JAMES KING, 50YRS M L-P A", [("JAMES KING", "NAME"), ("50YRS", "AGE"), ("M", "SEX")]),
    ("JULIA ANDERSON, 32 YEARS OLD, FEMALE", [("JULIA ANDERSON", "NAME"), ("32", "AGE"), ("FEMALE", "SEX")]),
    ("DAVID WHITE, 43YRS M L-P A", [("DAVID WHITE", "NAME"), ("43YRS", "AGE"), ("M", "SEX")]),
    ("EMILY JONES, 28 YEARS OLD, FEMALE", [("EMILY JONES", "NAME"), ("28", "AGE"), ("FEMALE", "SEX")]),
    ("MICHAEL BROWN, 37YRS M L-P A", [("MICHAEL BROWN", "NAME"), ("37YRS", "AGE"), ("M", "SEX")]),
    ("EVELYN YEBOAH, 60 YEARS OLD, FEMALE", [("EVELYN YEBOAH", "NAME"), ("60", "AGE"), ("FEMALE", "SEX")]),
    ("CHARLES OBIMPEH MENSAH, 38YRS Per 4cm W1024 / C512 CHEST Pos 1001 IMA 1001 S-Value: 449.000 Zoom factor", [("CHARLES OBIMPEH MENSAH", "NAME"), ("38YRS", "AGE")]),
    ("SARAH OWUSU, 25 YEARS OLD, FEMALE", [("SARAH OWUSU", "NAME"), ("25", "AGE"), ("FEMALE", "SEX")]),
    ("A NANA KWESI SARPONG, 14YRS", [("NANA KWESI SARPONG", "NAME"), ("14YRS", "AGE")]),
    ("THERESA KOOMSON, 70YRS L-P A AJSPECIALISTSCAN Ref:/Perf ID:FEB Study date: 16 F Study time: 08:09:48 4cm W1024 / C512 CHEST 1001 IMA 1001 S-Value: 2198.000 Zoom factor: x0.99", [("THERESA KOOMSON", "NAME"), ("70YRS", "AGE"), ("F", "SEX")]),
    ("JOHN DOE, 45 YEARS OLD, MALE", [("JOHN DOE", "NAME"), ("45", "AGE"), ("MALE", "SEX")]),
    ("A. Johanssen. Hosp no. 012345 Date. Today", [("A. Johanssen", "NAME")]),
    ("CHARLES OBIMPEH MENSAH, 38YRS Per 4cm W1024 / C512 CHEST Pos 1001 IMA 1001 S-Value: 449.000 Zoom factor", [("CHARLES OBIMPEH MENSAH", "NAME"), ("38YRS", "AGE")]),
     ("JAMES, KING, 50YRS M L-P A", [("JAMES KING", "NAME"), ("50YRS", "AGE"), ("M", "SEX")]),
    ("JULIA ANDERSON, 32 YEARS OLD, FEMALE", [("JULIA ANDERSON", "NAME"), ("32", "AGE"), ("FEMALE", "SEX")]),
    ("DAVID WHITE, 43YRS M L-P A", [("DAVID WHITE", "NAME"), ("43YRS", "AGE"), ("M", "SEX")]),
    ("EMILY JONES, 28 YEARS OLD, FEMALE", [("EMILY JONES", "NAME"), ("28", "AGE"), ("FEMALE", "SEX")]),
    ("MICHAEL BROWN, 37YRS M L-P A", [("MICHAEL BROWN", "NAME"), ("37YRS", "AGE"), ("M", "SEX")]),
    ("EVELYN YEBOAH, 60 YEARS OLD, FEMALE", [("EVELYN YEBOAH", "NAME"), ("60", "AGE"), ("FEMALE", "SEX")]),
    ("CHARLES,OBIMPEH,MENSAH,,38YRS Per 4cm W1024 / C512 CHEST Pos 1001 IMA 1001 S-Value: 449.000 Zoom factor", [("CHARLES OBIMPEH MENSAH", "NAME"), ("38YRS", "AGE")]),
    ("SARAH, OWUSU, 25 YEARS OLD, FEMALE", [("SARAH OWUSU", "NAME"), ("25", "AGE"), ("FEMALE", "SEX")]),
    ("A NANA KWESI SARPONG. 14YRS", [("NANA KWESI SARPONG", "NAME"), ("14YRS", "AGE")]),
    ("THERESA,KOOMSON,70YRS L-P A AJSPECIALISTSCAN Ref:/Perf ID:FEB Study date: 16 F Study time: 08:09:48 4cm W1024 / C512 CHEST 1001 IMA 1001 S-Value: 2198.000 Zoom factor: x0.99", [("THERESA KOOMSON", "NAME"), ("70YRS", "AGE"), ("F", "SEX")]),
    ("JOHN DOE, 45 YEARS OLD, MALE", [("JOHN DOE", "NAME"), ("45", "AGE"), ("MALE", "SEX")]),
    ("A. Johanssen. Hosp no. 012345 Date. Today", [("A. Johanssen", "NAME")]),
    ("CHARLES,OBIMPEH,MENSAH,,38YRS Per 4cm W1024 / C512 CHEST Pos 1001 IMA 1001 S-Value: 449.000 Zoom factor", [("CHARLES OBIMPEH MENSAH", "NAME"), ("38YRS", "AGE")]),
    ("SARAH, OWUSU, 25 YEARS OLD, FEMALE", [("SARAH OWUSU", "NAME"), ("25", "AGE"), ("FEMALE", "SEX")]),
    ("JAMES, KING, 50YRS M L-P A", [("JAMES KING", "NAME"), ("50YRS", "AGE"), ("M", "SEX")]),
    ("JULIA ANDERSON, 32 YEARS OLD, FEMALE", [("JULIA ANDERSON", "NAME"), ("32", "AGE"), ("FEMALE", "SEX")]),
    ("DAVID WHITE, 43YRS M L-P A", [("DAVID WHITE", "NAME"), ("43YRS", "AGE"), ("M", "SEX")]),
    ("EMILY JONES, 28 YEARS OLD, FEMALE", [("EMILY JONES", "NAME"), ("28", "AGE"), ("FEMALE", "SEX")]),
    ("MICHAEL BROWN, 37YRS M L-P A", [("MICHAEL BROWN", "NAME"), ("37YRS", "AGE"), ("M", "SEX")]),
    ("MARY JOHNSON, 55 YEARS OLD, FEMALE", [("MARY JOHNSON", "NAME"), ("55", "AGE"), ("FEMALE", "SEX")])
]


testset = [
    ("ALICE JONES, 35 YEARS OLD, FEMALE", [("ALICE JONES", "NAME"), ("35", "AGE"), ("FEMALE", "SEX")]),
    ("PETER SMITH, 42YRS M L-P A", [("PETER SMITH", "NAME"), ("42YRS", "AGE"), ("M", "SEX")]),
    ("LUCY WILLIAMS, 28 YEARS OLD, FEMALE", [("LUCY WILLIAMS", "NAME"), ("28", "AGE"), ("FEMALE", "SEX")]),
    ("ROBERT JOHNSON, 55YRS M L-P A", [("ROBERT JOHNSON", "NAME"), ("55YRS", "AGE"), ("M", "SEX")]),
    ("OLIVIA DAVIS, 31 YEARS OLD, FEMALE", [("OLIVIA DAVIS", "NAME"), ("31", "AGE"), ("FEMALE", "SEX")]),
    ("MICHELLE THOMAS, 48YRS F L-P A", [("MICHELLE THOMAS", "NAME"), ("48YRS", "AGE"), ("F", "SEX")]),
    ("BRIAN WILSON, 40 YEARS OLD, MALE", [("BRIAN WILSON", "NAME"), ("40", "AGE"), ("MALE", "SEX")]),
    ("EMMA HARRIS, 27YRS F L-P A", [("EMMA HARRIS", "NAME"), ("27YRS", "AGE"), ("F", "SEX")]),
    ("CHRISTOPHER CLARK, 50 YEARS OLD, MALE", [("CHRISTOPHER CLARK", "NAME"), ("50", "AGE"), ("MALE", "SEX")]),
    ("SOPHIA MARTIN, 33YRS F L-P A", [("SOPHIA MARTIN", "NAME"), ("33YRS", "AGE"), ("F", "SEX")])
]

In [ ]:
sentences = [ner_data[i][0] for i in range(len(ner_data))]
len(sentences)

167

In [ ]:
trainset, valset = train_test_split(ner_data, test_size = 0.2, random_state=42)

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def process_dataset(dataset):
    processed_data = []
    for sentence, entities in dataset:
        tokenized_input = tokenizer.tokenize(sentence)
        input_ids = tokenizer.convert_tokens_to_ids(tokenized_input)
        attention_mask = [1] * len(input_ids)
        token_type_ids = [0] * len(input_ids)

        entity_labels = ['O'] * len(tokenized_input)

        for entity, entity_type in entities:
            entity_tokens = tokenizer.tokenize(entity)
            entity_start = 0
            for i in range(len(tokenized_input)):
                if tokenized_input[i:i+len(entity_tokens)] == entity_tokens:
                    entity_labels[i] = 'B-' + entity_type
                    entity_start = i
                    for j in range(i+1, i+len(entity_tokens)):
                        entity_labels[j] = 'I-' + entity_type
                    break
        example = {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "token_type_ids": token_type_ids,
            "labels": entity_labels
        }
        processed_data.append(example)
    return processed_data

formatted_trainset = process_dataset(trainset)
formatted_valset = process_dataset(valset)


In [ ]:
label_map = {'O': 0, 'B-NAME': 1, 'I-NAME': 2, 'B-AGE': 3, 'I-AGE': 4, 'B-SEX': 5, 'I-SEX': 6}

In [ ]:
import torch
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from transformers import BertForTokenClassification, AdamW
from tqdm import tqdm

# Define the model configuration
model_name = "bert-base-uncased"
num_labels = 7  # Number of NER labels (excluding 'O')

# Load the pre-trained BERT model for token classification
model = BertForTokenClassification.from_pretrained(model_name, num_labels=num_labels)

# Set up the optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

# Convert the processed data into PyTorch DataLoader
train_dataloader = DataLoader(formatted_trainset, batch_size=16, shuffle=True, collate_fn=lambda x: dict(
    input_ids=pad_sequence([torch.tensor(example['input_ids']) for example in x], batch_first=True),
    attention_mask=pad_sequence([torch.tensor(example['attention_mask']) for example in x], batch_first=True),
    labels=pad_sequence([torch.tensor([label_map[label] for label in example['labels']]) for example in x], batch_first=True)
))
val_dataloader = DataLoader(formatted_valset, batch_size=32, shuffle=False, collate_fn=lambda x: dict(
    input_ids=pad_sequence([torch.tensor(example['input_ids']) for example in x], batch_first=True),
    attention_mask=pad_sequence([torch.tensor(example['attention_mask']) for example in x], batch_first=True),
    labels=pad_sequence([torch.tensor([label_map[label] for label in example['labels']]) for example in x], batch_first=True)
))

# Set the device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
num_epochs = 200
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0

    # Iterate over the training dataloader
    for batch in tqdm(train_dataloader, desc="Epoch {}".format(epoch + 1)):
        # Move batch tensors to the correct device
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Validation loop
    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for batch in tqdm(val_dataloader, desc="Validation"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            val_loss += loss.item()

    # Calculate average loss
    train_loss /= len(train_dataloader)
    val_loss /= len(val_dataloader)

    print("Epoch {} - Train Loss: {:.4f} - Val Loss: {:.4f}".format(epoch + 1, train_loss, val_loss))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: 

Epoch 1 - Train Loss: 1.2353 - Val Loss: 1.1424


Validation: 100%|██████████| 2/2 [00:03<00:00,  1.81s/it]


Epoch 2 - Train Loss: 0.5176 - Val Loss: 1.2770


Validation: 100%|██████████| 2/2 [00:04<00:00,  2.38s/it]


...

Validation: 100%|██████████| 2/2 [00:04<00:00,  2.21s/it]


Epoch 64 - Train Loss: 0.0055 - Val Loss: 0.0008


Validation: 100%|██████████| 2/2 [00:03<00:00,  1.81s/it]


Epoch 65 - Train Loss: 0.0048 - Val Loss: 0.0007


Epoch 66:  56%|█████▌    | 5/9 [00:41<00:32,  8.14s/it]

In [ ]:
formatted_testset = process_dataset(testset)

test_dataloader = DataLoader(formatted_testset, batch_size=32, shuffle=False, collate_fn=lambda x: dict(
    input_ids=pad_sequence([torch.tensor(example['input_ids']) for example in x], batch_first=True),
    attention_mask=pad_sequence([torch.tensor(example['attention_mask']) for example in x], batch_first=True),
    labels=pad_sequence([torch.tensor([label_map[label] for label in example['labels']]) for example in x], batch_first=True)
))

In [ ]:
from sklearn.metrics import accuracy_score, precision_score

# Evaluation loop for the test set
model.eval()
predictions = []
test_loss = 0.0
actual_flat = []

with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Testing"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits
        predicted_labels = torch.argmax(logits, dim=2)

        test_loss += loss.item()

        for i in range(predicted_labels.shape[0]):
            predicted_label_ids = predicted_labels[i][attention_mask[i] == 1].tolist()
            predicted_labels_str = [list(label_map.keys())[label_id] for label_id in predicted_label_ids]
            predictions.append(predicted_labels_str)
            actual_labels = process_dataset([testset[i]])
            actual_flat += actual_labels[0]['labels']

# Calculate average loss
test_loss /= len(test_dataloader)

# Convert the predicted labels and actual labels to flat lists
predicted_flat = [label for prediction in predictions for label in prediction]

# Calculate accuracy and precision
accuracy = accuracy_score(actual_flat, predicted_flat)
precision = precision_score(actual_flat, predicted_flat, average='micro')

# Print the test loss, accuracy, and precision
print("Test Loss:", test_loss)
print("Accuracy:", accuracy)
print("Precision:", precision)

# Print the predictions and actual labels for the test set
for i, prediction in enumerate(predictions):
    actual_labels = process_dataset([testset[i]])
    actual_labels = actual_labels[0]['labels']
    sentence = testset[i][0]

    print("Sentence:", sentence)
    print("Predicted Labels:", prediction)
    print("Actual Labels:", actual_labels)
    print()


Testing: 100%|██████████| 1/1 [00:00<00:00,  1.42it/s]

Test Loss: 0.003097509266808629
Accuracy: 1.0
Precision: 1.0
Sentence: ALICE JONES, 35 YEARS OLD, FEMALE
Predicted Labels: ['B-NAME', 'I-NAME', 'O', 'B-AGE', 'O', 'O', 'O', 'B-SEX']
Actual Labels: ['B-NAME', 'I-NAME', 'O', 'B-AGE', 'O', 'O', 'O', 'B-SEX']

Sentence: PETER SMITH, 42YRS M L-P A
Predicted Labels: ['B-NAME', 'I-NAME', 'O', 'B-AGE', 'I-AGE', 'I-AGE', 'B-SEX', 'O', 'O', 'O', 'O']
Actual Labels: ['B-NAME', 'I-NAME', 'O', 'B-AGE', 'I-AGE', 'I-AGE', 'B-SEX', 'O', 'O', 'O', 'O']

Sentence: LUCY WILLIAMS, 28 YEARS OLD, FEMALE
Predicted Labels: ['B-NAME', 'I-NAME', 'O', 'B-AGE', 'O', 'O', 'O', 'B-SEX']
Actual Labels: ['B-NAME', 'I-NAME', 'O', 'B-AGE', 'O', 'O', 'O', 'B-SEX']

Sentence: ROBERT JOHNSON, 55YRS M L-P A
Predicted Labels: ['B-NAME', 'I-NAME', 'O', 'B-AGE', 'I-AGE', 'I-AGE', 'B-SEX', 'O', 'O', 'O', 'O']
Actual Labels: ['B-NAME', 'I-NAME', 'O', 'B-AGE', 'I-AGE', 'I-AGE', 'B-SEX', 'O', 'O', 'O', 'O']

Sentence: OLIVIA DAVIS, 31 YEARS OLD, FEMALE
Predicted Labels: ['B-NAME'

In [ ]:
from transformers import BertForTokenClassification, BertConfig

# Save the model
output_dir = "./"
model.save_pretrained(output_dir)

# Load the saved model for inference
model = BertForTokenClassification.from_pretrained(output_dir, num_labels=num_labels)

# New sentences for inference
sentences = ["KOFI MANU IS 14YRS OLD.", "ADRIAN 43YRS F."]

# Tokenize and convert sentences to input features
encoded_inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")

# Move input tensors to the device
input_ids = encoded_inputs["input_ids"].to(device)
attention_mask = encoded_inputs["attention_mask"].to(device)

# Inference
model.eval()
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    predicted_labels = torch.argmax(logits, dim=2)

# Convert predicted labels to string labels
predicted_labels = [list(label_map.keys())[label_id] for label_id in predicted_labels[0]]

# Print the predicted labels for each sentence
for sentence, labels in zip(sentences, predicted_labels):
    print("Sentence:", sentence)
    print("Predicted Labels:", labels)
    print()

Sentence: KOFI MANU IS 14YRS OLD.
Predicted Labels: B-NAME

Sentence: ADRIAN 43YRS F.
Predicted Labels: I-NAME

